## 卷积神经网络 循环神经网络

In [ ]:
# 卷积神经网络非常适合处理在空间上有一定关系的数据
# 循环神经网络非常适合处理在时间上有一定关系的数据

# 卷积神经网络可以理解为空间维度上的参数共享
# 循环神经网络可以理解为时间维度上的参数共享

#### 建立的出发点：参数共享。

In [ ]:
# 全连接神经网络最大的问题就是参数个数过多，模型不知道该怎么学习参数。图像（空间关联）、文本（时间关联）
# 全连接神经网络过于简单粗暴，在很多问题上就容易出现过拟合现象。
# 参数集中一下

# 卷积神经网络可以处理任意size的输入图像，前提是不引入任何全连接层的情况
# 循环神经网络可以处理任意长度的输入数据。